In [3]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

In [4]:
df = pd.read_csv("C:/Users/nwalker018/Desktop/Python/bills/bills_with_sponsor.csv")
df1 = pd.read_csv("C:/Users/nwalker018/Desktop/Python/bills/bill_cosponsorship.csv")
df2 = pd.read_csv("C:/Users/nwalker018/Desktop/Python/bills/bill_details.csv")
df3 = pd.DataFrame(df2)

In [5]:
df.head()

,id,first_name,last_name,bill_number,sponsor_state,sponsor_party
0,M001165,Kevin,McCarthy,H.RES.2,CA,R
1,S000250,Pete,Sessions,H.CON.RES.1,TX,R
2,G000553,Al,Green,H.CON.RES.3,TX,D
3,C001053,Tom,Cole,H.CON.RES.2,OK,R
4,J000032,Sheila,Jackson Lee,H.RES.13,TX,D


In [6]:
df3.head()

,bill_number,title,primary_subject
0,H.RES.2,To inform the Senate that a quorum of the Hous...,Congress
1,H.CON.RES.1,Regarding consent to assemble outside the seat...,Congress
2,H.CON.RES.3,Recognizing former United States Federal Judge...,"Civil Rights and Liberties, Minority Issues"
3,H.CON.RES.2,To authorize the use of United States Armed Fo...,International Affairs
4,H.RES.13,Expressing the sense of the House of Represent...,Emergency Management


In [7]:
df2.head()

,bill_number,title,primary_subject
0,H.RES.2,To inform the Senate that a quorum of the Hous...,Congress
1,H.CON.RES.1,Regarding consent to assemble outside the seat...,Congress
2,H.CON.RES.3,Recognizing former United States Federal Judge...,"Civil Rights and Liberties, Minority Issues"
3,H.CON.RES.2,To authorize the use of United States Armed Fo...,International Affairs
4,H.RES.13,Expressing the sense of the House of Represent...,Emergency Management


In [8]:
df2[:2]

,bill_number,title,primary_subject
0,H.RES.2,To inform the Senate that a quorum of the Hous...,Congress
1,H.CON.RES.1,Regarding consent to assemble outside the seat...,Congress


In [9]:
import re, math
from collections import Counter

In [10]:
WORD = re.compile(r'\w+')

In [11]:
def cosine_sim(a, b):
    """Takes 2 vectors a, b and returns the cosine similarity according 
    to the definition of the dot product"""
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

In [12]:
def makeVector(self, wordString):
        """ @pre: unique(vectorIndex) """

        #Initialise vector with 0's
        vector = [0] * len(self.vectorKeywordIndex)
        wordList = self.parser.tokenise(wordString)
        wordList = self.parser.removeStopWords(wordList)
        for word in wordList:
                vector[self.vectorKeywordIndex[word]] += 1; #Use simple Term Count Model
        return vector

In [13]:
#df3.iloc[:,[0,2]]
def sparse_matrix(df):
    
    df['bill_number'] = df['bill_number'].astype("category")
    df['primary_subject'] = df['primary_subject'].astype("category")
    
    out = coo_matrix(df['bill_number'].cat.codes.copy(),
                     df['primary_subject'].cat.codes.copy())
    return out

In [14]:
sparse_matrix(df3)

<1x7977 sparse matrix of type '<class 'numpy.int16'>'
	with 7976 stored elements in COOrdinate format>

<h2> BM25 Matching </h2>
- BM - Best Matching
- ***Is one of the best probablistic weighting schemes***
- is a bag-of-words retrieval function that ranks a set of documents based on the query terms appearing in each document
![alt text](Desktop/BM25.png)

<div style='float: center'>
  <img style='width: 600px' src="Desktop/BM25.PNG"></img>
</div>

In [19]:
def bm25_weight(X, K1=3000, B=0.8):
    """ Weighs each row of the sparse matrix of the data by BM25 weighting """
    # calculate idf per term (user)
    X = coo_matrix(X)
    N = x.shape[0]
    idf = np.log(float(N) / (1 + np.bincount(X.col)))
    
    # calculate length_norm per document 
    row_sums = np.ravel(X.sum(axis=1))
    avg_length = row_sums.mean()
    length_norm = (1.0 - B) + B * row_sums / avg_length
    
    # weight matrix rows by BM25
    X.data = X.data * (K1 + 1.0) / (k1 * length_norm[x.row] + X.data) * idf[X.col]
    return X

In [20]:
class TopRelated(object):
    def __init__(self, congress_factors):
        # fully normalize congress_factors, so we can compare with only the dot product
        norms = np.linalg.norm(congress_factors, axis=-1)
        self.factors = congress_factors / norms[:, np.newaxis]
        
    def get_related(self, congressID, N=10):
        scores = self.factors.dot(self.factors[congressID])
        best = numpy.argpartition(scores, -N)[-N:]
        return sorted(zip(best, scores[best]), key=lambda x: -x[1])